### MountainCar-v0 Environment

In [12]:
import gymnasium as gym
env = gym.make('MountainCar-v0', render_mode='rgb_array')
obs, info = env.reset()
import mediapy

frames = []
c = 0
while True:
    c += 1
    # sample random action
    # action = env.action_space.sample()
    action = 1
    obs, rewards, terms, truncs, info = env.step(action)
    done = terms or truncs
    img = env.render()
    frames.append(img)
    if done:
        break
env.close()
mediapy.show_videos({'MountainCar-V0-RandomAgent':frames})

MountainCar-V0-RandomAgent This browser does not support the video tag.


In [9]:
env.action_space, action,type(action)

(Discrete(3), 1, numpy.int64)

### Minigrid with 5x5 map, no walls / maze at all

In [2]:
import minigrid
import gymnasium as gym
import mediapy

name = "MiniGrid-Empty-5x5-v0"
env = gym.make(name, render_mode="rgb_array")
observation, info = env.reset(seed=42)
frames = []
for c in range(1000):
   # action = policy(observation)  # User-defined policy function
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    frames.append(env.render())
    if terminated or truncated:
        observation, info = env.reset()
env.close()

mediapy.show_videos({name: frames})

MiniGrid-Empty-5x5-v0 This browser does not support the video tag.


In [13]:
import minigrid
import gymnasium as gym
import mediapy

name = "MiniGrid-FourRooms-v0"
env = gym.make(name, render_mode="rgb_array")
observation, info = env.reset(seed=42)
frames = []
for c in range(1000):
   # action = policy(observation)  # User-defined policy function
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    frames.append(env.render())
    if terminated or truncated:
        observation, info = env.reset()
env.close()

mediapy.show_videos({name: frames})

MiniGrid-FourRooms-v0 This browser does not support the video tag.


In [14]:
env.action_space

Discrete(7)

In [2]:
# One-way door construction in minigrid will be interesting -- this will actually help evaluating
# -- the quqsi-metric function's power, mountain car is meaningful since gravity is in play, too (I guess)

#### Unfortunately, MiniWorld requires lesser than python 3.11 version (newest not compatible)

In [4]:


import gymnasium as gym
env = gym.make("MiniWorld-OneRoom-v0")
# observation, info = env.reset(seed=42)
# for _ in range(1000):
#     action = policy(observation)  # User-defined policy function
#     observation, reward, terminated, truncated, info = env.step(action)

#     if terminated or truncated:
#         observation, info = env.reset()
# env.close()

NameNotFound: Environment `MiniWorld-OneRoom` doesn't exist. Did you mean: `MiniGrid-LockedRoom`?

In [10]:
import gymnasium as gym
import mediapy as media
name = 'roundabout-v0'
env = gym.make(name, render_mode='rgb_array')
frames = []
obs, info = env.reset()
done = truncated = False
while not (done or truncated):
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
    frames.append(env.render())
media.show_videos({name: frames}, fps=6)

roundabout-v0 This browser does not support the video tag.
